# Ground-Based Measurements
(10 minutes)

Learning Objectives: 
* Become familiar with which measurements are core validation/calibration data sets
* Know the sampling strategy and naming conventions for each campaign
* Learn where to find and search for SnowEx data

## Data Descriptions
### What ground-based data sets are central for all field campagins? 

<figure>
<img src='content/02_pits_and_depths.png'>
<figcaption algin = "left"> <b>Figure 1.</b> Ground-based measurements central to the SnowEx campaigns. (L) Snow Pit: Observer measures the stratigraphy in the snow pit. (M) Depth Transects: Observer uses the MagnaProbe to measure snow depths. (R). Interval Boards: Observer stomps down the snow to place the interval board for another weekly long measurement cycle.     
<figure>


___
### What is a **snow pit**? <span style="color:gray">  ⇨ 'Layer data' & 'Site data' in the SnowEx database
A snow pit is a vertical profile of snow from the snow surface to the ground.
This is where we collect a suite of ground observations that aid instrument and model development.

### What are **depth transects (spirals)**? or simply **depths** <span style="color:gray">  ⇨ 'Point data' in the SnowEx database
These are point meausurements that tell you how deep the snow is at georeferenced locations.
There are several instruments available to capture snow depth, some of which are an avalanche probe, MagnaProbe, or Mesa2.

### What is an **interval board**? <span style="color:gray">   ⇨ soon to be 'Point data' in the SnowEx database
An interval board is a 2x2" board with a vertical pole attached at the center.
The board surface is our reference to track the amount of new snow that falls between field visits.
The board remains fixed for the season, and gets cleared each visit.
    
___

### Additional detail on what's in a snow pit
<figure>
<img src='content/02_pit-measurements.png'>
<figcaption algin = "left"> <b>Figure 2.</b> Description of parameters that come from a single snow pit visit. The colored ovals on the images correspond to the colored measurements on the left side.
<figure>

    
Explore more detail in the [2020 Experimental Plan](https://snow.nasa.gov/sites/default/files/NASA_SnowEx_Experiment_Plan_v15_draft.pdf)

### Data Collection
Field note photos from one pit visits. We call these __Pit Sheets__.

<figure>
<img src='content/02_SnowEx20_SnowPits_GMIOP_20200212_2C12_book1_v01.jpg'>
    
<img src='content/02_stratCodes.png'>
    
<img src='content/02_SnowEx20_SnowPits_GMIOP_20200212_2C12_book2_v01.jpg'>
<figcaption algin = "left"> <b>Figure 3.</b> Front and back of Pit Sheets field forms. These have largely remained the same throughout the different campaigns. The middle image is a reference for the codes used in the Stratigraphy measurements. 
<figure>

### Why do we care about observations from snow pits and other measurements? 
* 90% because we need to validate what we see from airborne and satellite sensors!!
* 10% because we are geoscientists that want to winter recreate!!

(You can argue those percentages over our movement / coffee break)
<figure>
<img src='content/02_reasons-to-measure.png'>
<figcaption algin = "left"> <b>Figure 4.</b> Examples of snow properties and features that are senstive to modeling and remote sensing signals. 
<figure>

### What are some other data sets?

## Sampling Strategy and Naming Conventions
### 2020 Grand Mesa IOP

<figure>
<img src='content/02_matrix.png'>
<figcaption algin = "left"> <b>Figure 5.</b> Nine-member snow and vegetation matrix used to sample and name snow pits during the Grand Mesa IOP campaign 
<figure>
    
**Vegetation Classes**
- 1-3 = treeless (0% tree cover)
- 4-6 = sparse (1-30% tree cover)
- 7-9 = dense (31-100% tree cover)

**Snow Depth Classes**
- 1, 4, 7 = shallow (<90cm)
- 2, 5, 8 = medium (90-122cm)
- 3, 6, 9 = deep (>122cm)

**Flight Lines**
- 'C' = Crossline
- 'N' = North
- 'S' = South
    
<figure>
<img src='content/02_GMpitMap.jpg'>
<figcaption algin = "left"> <b>Figure 6.</b> Grand Mesa IOP 2020 sampling design. The rectangles are the lidar, UAVSAR, SWESARR, Thermal IR, and Gamma flight lines. The point markers are the locations of snow pits. 
<figure>

### 2020 & 2021 Time Series 

- Location     ⇨ Regional Location with existing long-term monitoring infrastructure.
- Site         ⇨ Averaged unique snow pit site to represent the progression of Time Series snow pits.
- PitID        ⇨ 6-letter code unique to all snow pit sites; 2-letter state, 2-letter Location; 2-letter site name (e.g **Id**aho, **B**oise **R**iver, **B**ogus **U**pper = IDBRBU
- Unique PitID ⇨ 6-letter site code and timestamp (e.g. format: XXXXXX_yyyymmdd_hhmm, example: IDBRBU_20200116_1345)

**Table 1. (Preliminary) Comprehensive Site Summary for 2020 and 2021 Time Series Campaign**
|State|Location                  |Site              |PitID |Latitude|Longitude |Pit Count|Snow Class|
|-----|--------------------------|------------------|------|--------|----------|---------|----------|
|CA   |American River Basin      |Caples Lake       |CAAMCL|38.71029|-120.04186|7        |3         |
|CA   |Mammoth Lakes             |CUES              |CAMLCP|37.64324|-119.02906|9        |5         |
|CA   |Mammoth Lakes             |Panorama Dome     |CAMLPD|37.61964|-119.00029|6        |6         |
|CA   |Sagehen Creek             |Forest            |CASHFO|39.42956|-120.24211|7        |6         |
|CA   |Sagehen Creek             |Open              |CASHOP|39.43037|-120.23982|7        |6         |
|CA   |Sagehen Creek             |Tower 4           |CASHT4|39.42216|-120.29898|6        |3         |
|CO   |Cameron Pass              |Cameron Peak      |COCPCP|40.56381|-105.86761|16       |6         |
|CO   |Cameron Pass              |Joe Wright        |COCPJW|40.52406|-105.89345|8        |2         |
|CO   |Cameron Pass              |Michigan River    |COCPMR|40.51864|-105.89193|22       |2         |
|CO   |East River                |Aspen             |COERAP|38.92529|-106.97109|9        |6         |
|CO   |East River                |Forest 12         |COER12|38.92973|-106.97824|12       |2         |
|CO   |East River                |Forest 13         |COER13|38.92943|-106.97777|9        |6         |
|CO   |East River                |Forest 14         |COER14|38.92894|-106.97757|12       |6         |
|CO   |East River                |Gothic            |COERGT|38.95925|-106.99053|9        |6         |
|CO   |East River                |Irwin Barn        |COERIB|38.88812|-107.10796|13       |2         |
|CO   |East River                |Open 2            |COERO2|38.92661|-106.97848|9        |2         |
|CO   |East River                |Open 4            |COERO4|38.92627|-106.97907|12       |2         |
|CO   |East River                |Open 6            |COERO6|38.92669|-106.97988|9        |2         |
|CO   |East River                |Trench 13         |COERTR|38.92920|-106.97770|5        |6         |
|CO   |East River                |Upper             |COERUP|38.93331|-106.98994|12       |2         |
|CO   |Fraser Experimental Forest|JPL 1             |COFEJ1|39.90556|-105.88282|35       |6         |
|CO   |Fraser Experimental Forest|JPL 2             |COFEJ2|39.90612|-105.88255|26       |6         |
|CO   |Fraser Experimental Forest|SNB 1             |COFEB1|39.90699|-105.87786|23       |6         |
|CO   |Fraser Experimental Forest|SNB 2             |COFEB2|39.90703|-105.87904|18       |6         |
|CO   |Grand Mesa                |County Line Open  |COGMCO|39.03053|-108.03220|12       |2         |
|CO   |Grand Mesa                |County Line Tree  |COGMCT|39.03223|-108.03438|10       |2         |
|CO   |Grand Mesa                |Mesa West Open    |COGMWO|39.03399|-108.21390|10       |2         |
|CO   |Grand Mesa                |Mesa West Trees   |COGMWT|39.03282|-108.21291|7        |2         |
|CO   |Grand Mesa                |Skyway Open       |COGMSO|39.04511|-108.06209|21       |2         |
|CO   |Grand Mesa                |Skyway Tree       |COGMST|39.04565|-108.06333|18       |2         |
|CO   |Niwot Ridge               |C1                |CONWC1|40.03317|-105.54616|9        |6         |
|CO   |Niwot Ridge               |Forest Flat       |CONWFF|40.03186|-105.54667|12       |6         |
|CO   |Niwot Ridge               |Forest North      |CONWFN|40.03337|-105.55380|11       |2         |
|CO   |Niwot Ridge               |Forest South      |CONWFS|40.04021|-105.55701|13       |2         |
|CO   |Niwot Ridge               |Open Flat         |CONWOF|40.03579|-105.55205|15       |6         |
|CO   |Niwot Ridge               |Saddle            |CONWSA|40.05497|-105.59065|7        |1         |
|CO   |Senator Beck              |Senator Beck      |COSBSB|37.90705|-107.72626|5        |1         |
|CO   |Senator Beck              |Swamp Angel       |COSBSA|37.90714|-107.71121|19       |2         |
|ID   |Boise River Basin         |Banner Open       |IDBRBO|44.30456|-115.23601|11       |6         |
|ID   |Boise River Basin         |Banner Snotel     |IDBRBS|44.30362|-115.23457|23       |6         |
|ID   |Boise River Basin         |Bogus Lower       |IDBRBL|43.76434|-116.10660|17       |6         |
|ID   |Boise River Basin         |Bogus Lower Trees |IDBRBT|43.76057|-116.08979|13       |6         |
|ID   |Boise River Basin         |Bogus Upper       |IDBRBU|43.75881|-116.09019|11       |6         |
|ID   |Boise River Basin         |LDP Open          |IDBRLO|43.73707|-116.12179|19       |6         |
|ID   |Boise River Basin         |LDP Tree          |IDBRLT|43.73634|-116.12053|17       |6         |
|ID   |Boise River Basin         |Mores Creek Summit|IDBRMC|43.94735|-115.67666|3        |6         |
|MT   |Central Ag Research Center|SnowEx-1          |MTCASX|47.05836|-109.95688|4        |5         |
|MT   |Central Ag Research Center|Wheat             |MTCAWH|47.05942|-109.95176|1        |5         |
|MT   |Central Ag Research Center|Wx                |MTCAWX|47.06062|-109.95693|4        |5         |
|NM   |Jemez River               |BA Flux Tower     |NMJRBA|35.88859|-106.53184|8        |5         |
|NM   |Jemez River               |HQ Met Station    |NMJRHQ|35.85791|-106.52137|8        |5         |
|UT   |Little Cottonwood Canyon  |Alta Collins      |UTLCAC|40.57210|-111.62997|17       |2         |
|UT   |Little Cottonwood Canyon  |Atwater           |UTLCAW|40.59125|-111.63759|20       |5         |

Note - This table is a preliminary comprehensive site summary of the 2020 and 2021 Time Series campaigns. If a site was repeated (e.g. Cameron Pass, Michigan River) then the Pit Count column is a sum of the two years). This is preliminary because we are still adjusting and correcting some of the spatial data after confirming locations and sampling designs with site leads. The Snow Class column was extracted from the 300 m Snow Classification data set (Liston, 2021 and can be accessed [here](https://nsidc.org/data/NSIDC-0768/versions/1) at NSIDC). See the previous notebook to characterize the Snow Classes. This table does not include about ~50 snow pits that were part of the 2021 TS campaign from the Fraser Experiemental Forest becuase of the modified spatial sampling design, but will be included in the published data set, and the table will be updated to reflect those additional snow pits. 

This table can be accessed as a geojson from [Megan's github gists](https://gist.github.com/meganmason/dde13c46a90875e364b1c25a31bff1d8) 

To use this as a geojson in your project:
1. Navigate to the link above
2. Check out the map of data (zoom in, zoom out, etc.)
3. Select the 'Raw' button from the top right corner
4. Copy/Paste the URL (it's longer now)
5. Use the code block below to import

In [ ]:
import geopandas as gpd
import requests
url = "https://gist.githubusercontent.com/meganmason/dde13c46a90875e364b1c25a31bff1d8/raw/d361e1a56eeb867bd89c87b4dc13ed431ffff6f6/SNEX_TS_SP_avg_prelim_Hackweek.geojson"
gdf = gpd.read_file(url)
gdf.head()

### Where can I find data?

#### 1. SnowEx data sets are stored and archived at the National Snow and Ice Data Center (NSIDC) in Boulder, Colorado. 

**Table 2. Summary of Core Data Sets with DOI links for applicable data sets**
<table id="t01">
<tbody>
<tr>
<th>Core Data Sets</th>
<th>Parameters Available</th>
<th>Instruments</th>
<th>NSIDC DOI</th>
<hr>
</tr>
<tr>
<td>Snow Pits</td>
<td>Depth, Density, Temp., LWC, Stratigraphy,<br>Grain Size, Wetness, Hand Hardness,<br>Qualitative Environment, SWE</td>
<td>Standard pit kit</td>
<td><a href='https://nsidc.org/data/SNEX17_SnowPits/versions/1'>SNEX17_SnowPits_GMIOP</a> 
<br>
<a href='https://nsidc.org/data/SNEX20_GM_SP/versions/1'>SNEX20_SnowPits_GMIOP</a>
<br>
SNEX20_TS_SP (in review)
<br>
SNEX21_TS_SP (future pub.)<td>
</tr>
<tr>
<td>Depth Transects</td>
<td>Snow Depth - various sampling designs</td>
<td>MagnaProbe, Mesa2, Pit Ruler</td>
<td><a href='https://nsidc.org/data/SNEX17_SD/versions/1'>SNEX17_SD</a> 
<br>
<a href='https://nsidc.org/data/SNEX20_SD/versions/1'>SNEX20_SD</a>
<br>
SNEX20_TS_SD (future pub.)
<br>
SNEX21_TS_SD (future pub.)<td>
</tr>
<tr>
<td>Interval Boards</td>
<td>New snow, SWE, density</td>
<td>Ruler and spring scale</td>
<td>SNEX20_TS_IB (in prep.)
<br>
SNEX21_TS_IB (future pub.)<td></td>
</tr>
<tr>
</tr>
<tbody>
<table> 
<hr>

_Visit [NSIDC's SnowEx Overview page](https://nsidc.org/data/snowex) to see a full table of all available [2017](https://nsidc.org/data/snowex/data_summaries), [2020](https://nsidc.org/data/snowex/data_summaries), and [2021](https://nsidc.org/data/snowex/data_summaries) products._
    
 <figure>
<img src='content/02_data-package.png'>
<figcaption algin = "left"> <b>Figure 7.</b> Representative snow pit data package for an individual pit visit.  
<figure>

#### 2. SnowEx SQL Database (preview in next tutorial!)

## Recap

* You know which measurements we have a LOT of and are consistant between multiple field campaigns
* You are aware of the basic sampling strategy for the two main campaign types
* You have a few tools at your finger tips to access and learn more about the data

## Share Your Thoughts
### Group Mind Map Activity 

1. Navigate to this [Google Slides](https://docs.google.com/presentation/d/1Vht7MCePDKQsts_baUAvtQZBkRsldtMzeOmztkiNAKA/edit#slide=id.g13bf53aeb4c_0_0) page to complete the Mind Map activity
2. Wait to be assigned a table number
3. Add your thoughts on data applications and research topics now that you know what measurements we have and roughly where they occur.
